# Laboratorio 5 - Parte 1

### Máquinas de Vectores de Soporte

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante:  Santiago Alexis Sanchez Zuleta
#### Segundo Integrante: Raul Andres Ramirez Gomez


In [110]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")
%matplotlib inline

## Ejercicio 1

En este ejercicio deben resolver un problema de clasificación multi-clase usando una SVM. Teniendo en cuenta que la formulación original de la SVM sólo permite resolver problemas bi-clase, deben comparar los resultados obtenidos usando una estrategia **Uno vs Uno** con una estrategia **Uno vs El resto**.

Cargamos los datos:

In [111]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Consutar el manejo de la librería sklearn para entrenar un modelos SVM en: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. Para el caso multiclase la librería ya tiene implementada la estrategia **Uno vs Uno**, así que en este caso sólo deben llamar correctamente los métodos. Complete el siguiente código usando un clasificador basado en SVM:

In [112]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold


#Validamos el modelo
Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    modelo = SVC(gamma='scale', decision_function_shape='ovr')
    modelo.fit(Xtrain, Ytrain)
    
    #Validación
    Ytrain_pred = modelo.predict(Xtrain)
    Yest = modelo.predict(Xtest)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
    j += 1
        
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9861111111111112+-0.004811252243246912


Realice los experimientos necesarios para llenar la siguiente tabla:

In [113]:
def simulacion(Kernel,C,gamma):
    #Validamos el modelo
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    percent=np.zeros(4)
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if (Kernel == 'lineal'):
            modelo = SVC(kernel='linear', C=C)
        else:
            modelo = SVC(kernel=Kernel, C=C, gamma=gamma)
        
        modelo.fit(Xtrain, Ytrain)

        #Validación
        Ytrain_pred = modelo.predict(Xtrain)
        Yest = modelo.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        percent[j] = len(modelo.support_vectors_) / len(Xtrain)
        j += 1
            

    print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    print('Porcentaje de los modelos = ' + str(np.mean(percent)))
    return np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(percent)
   #return str(np.mean(EficienciaVal)),np.std(EficienciaVal),str(np.mean(percent))

In [114]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)

for i in df_types.index:
    Eficiencia,confianza,Vectores= simulacion(i[0], i[1], i[2])
    df_types["Eficiencia en validacion"][i] = Eficiencia
    df_types["Intervalo de confianza"][i] = confianza
    df_types["% de Vectores de Soporte"][i] = Vectores


#df_types["Eficiencia en validacion"][3] = "0.97077"
#df_types["Intervalo de confianza"][3] = "0.01548"
#df_types["% de Vectores de Soporte"][3] = "0.2620"
#df_types.sort_index(inplace=True)
#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

Eficiencia durante el entrenamiento = 0.9203703703703704+-0.025793311624415057
Eficiencia durante la validación = 0.7805555555555556+-0.06922186552431732
Porcentaje de los modelos = 0.999537037037037
Eficiencia durante el entrenamiento = 0.9967592592592593+-0.002018008770157708
Eficiencia durante la validación = 0.9694444444444446+-0.02763853991962833
Porcentaje de los modelos = 0.6111111111111112
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9791666666666667+-0.016839382851364076
Porcentaje de los modelos = 0.2847222222222222
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9750000000000001+-0.011453071182271271
Porcentaje de los modelos = 0.26435185185185184
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9750000000000001+-0.011453071182271271
Porcentaje de los modelos = 0.26435185185185184
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.975

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [115]:
df_types

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                  0.780556              0.0692219   
       0.010   0.00                  0.969444              0.0276385   
       0.100   0.00                  0.979167              0.0168394   
       1.000   0.00                     0.975              0.0114531   
       10.000  0.00                     0.975              0.0114531   
       100.000 0.00                     0.975              0.0114531   
rbf    0.001   0.01                  0.313889               0.104268   
               0.10                  0.256944             0.00605403   
               1.00                  0.254167             0.00240563   
       0.010   0.01                  0.313889               0.104268   
               0.10                  0.256944             0.00605403   
               1.00                  0.254167             0.00240563   
       0.100   0.01                    0.8375              0.0421957   
               0.10                  0.256944             0.00605403   
               1.00                  0.254167             0.00240563   
       1.000   0.01                  0.981944              0.0138193   
               0.10                  0.809722              0.0431003   
               1.00                  0.254167             0.00240563   
       10.000  0.01                  0.990278             0.00821678   
               0.10                  0.815278              0.0457069   
               1.00                  0.256944             0.00460642   
       100.000 0.01                  0.990278             0.00821678   
               0.10                  0.815278              0.0457069   
               1.00                  0.256944             0.00460642   

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                  0.999537  
       0.010   0.00                  0.611111  
       0.100   0.00                  0.284722  
       1.000   0.00                  0.264352  
       10.000  0.00                  0.264352  
       100.000 0.00                  0.264352  
rbf    0.001   0.01                  0.999537  
               0.10                  0.999537  
               1.00                  0.999074  
       0.010   0.01                  0.999537  
               0.10                         1  
               1.00                         1  
       0.100   0.01                  0.999537  
               0.10                         1  
               1.00                         1  
       1.000   0.01                  0.643056  
               0.10                  0.972222  
               1.00                         1  
       10.000  0.01                  0.574537  
               0.10                  0.972222  
               1.00                         1  
       100.000 0.01                  0.574537  
               0.10                  0.972222  
               1.00                         1

## Ejercicio 2

Cree dos funciones, una para entrenar un conjunto de modelos bajo la estrategia Uno vs el resto, usando como clasificador base una SVM. La segunda función debe usar el conjunto de modelos entrenados, y clasificar un conjunto de muestras de validación.

#### Nota: No se permite el uso de librerías para resolver este ejercicio

In [156]:
def TrainSVM_OnevsRest(kernel, C , gamma, X_train, Y_train):
    classes = np.unique(Y_train).size
    Y= np.zeros(len(Y_train))
    simulationsMetod= []
    #We see each class and their coincidence
    for i in range(0,classes):
        Y[Y_train == i] = 1
        Y[Y_train != i] = -1
        mod= SVC(C=C, kernel=kernel, gamma=gamma, probability = True) 
        mod.fit(X_train,Y)
        simulationsMetod.append(mod)
        
    return simulationsMetod


In [157]:
    
def ValidaSVM_OnevsRest(simulations,tests):
    predictions=[]
    #Comparison between data set designed to tests and the silmulations
    for i in tests:
        models_results=[]
        for j in simulations:
            probability = j.predict_proba([i])
            models_results.append(probability[0][1])      
        model=np.argmax(models_results)
        predictions.append(model)
        
    return np.array(predictions)

## Ejercicio 3

Use las funciones definidas en el punto anterior para llevar a cabo la prueba de simulación con el mismo conjunto de datos del punto 1.

In [158]:
Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    
    models =TrainSVM_OnevsRest('linear', 1, 0.01, Xtrain, Ytrain)
    
    
    #Validación
    Ytrain_pred = ValidaSVM_OnevsRest(models,Xtrain)
    Yest = ValidaSVM_OnevsRest(models,Xtest)
    #Ef=np.mean(EficienciaVal)
    #print(Ef)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
    j += 1
        
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación  prueba= ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación  prueba= 0.9777777777777779+-0.015214515486254602


Realice los experimientos necesarios para llenar la siguiente tabla:

In [162]:

def simulacion2(Kernel,C,gamma):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    percent=np.zeros(4)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if (Kernel == 'lineal'):
            Kernel='linear'
        if(gamma==0):
            gamma = 'auto'
        percent[j]=0  
        mod= TrainSVM_OnevsRest(Kernel, C, gamma, Xtrain, Ytrain)
        for simulation in mod:
            percent[j] = percent[j] + (simulation.support_vectors_.shape[0])/Xtrain.shape[0]

        #Validación
        Ytrain_pred = ValidaSVM_OnevsRest(mod,Xtrain)
        Yest = ValidaSVM_OnevsRest(mod,Xtest)


        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean((np.array(Ytrain_pred)).ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean((np.array(Yest)).ravel() == Ytest.ravel())
        #Ef=np.mean(EficienciaVal)
        #print(Ef)
        j += 1 
  

    #print("Kernel: "+ str(Kernel)+ " C :"+ str(C) + " gamma:" + str(gamma))
    #print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
   #print('Eficiencia durante la validación metodo = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    
    return np.mean(EficienciaVal),np.std(EficienciaVal),np.mean(percent)

In [163]:
import pandas as pd
import qgrid
df_types2 = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types2["Eficiencia en validacion"] = ""
df_types2["Intervalo de confianza"] = ""
df_types2["% de Vectores de Soporte"] = ""
df_types2.set_index(['Kernel','C','gamma'], inplace=True)
for r in df_types2.index:
    Eficiencia2,confianza2,Vectores2= simulacion2(r[0], float(r[1]),float(r[2]))
    df_types2["Eficiencia en validacion"][r] = Eficiencia2
    df_types2["Intervalo de confianza"][r] = confianza2
    df_types2["% de Vectores de Soporte"][r] = Vectores2
#df_types["Eficiencia en validacion"][3] = "0.97633"
#df_types["Intervalo de confianza"][3] = "0.01837"
#df_types["% de Vectores de Soporte"][3] = "0.2778"
#df_types.sort_index(inplace=True)
#qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
#qgrid_widget

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [161]:
df_types2

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                  0.954167              0.0381629   
       0.010   0.00                    0.9625              0.0341056   
       0.100   0.00                  0.976389              0.0201748   
       1.000   0.00                  0.977778              0.0152145   
       10.000  0.00                  0.977778              0.0152145   
       100.000 0.00                  0.977778              0.0152145   
rbf    0.001   0.01                  0.961111              0.0291336   
               0.10                  0.963889              0.0168966   
               1.00                  0.284722              0.0181621   
       0.010   0.01                  0.965278              0.0315189   
               0.10                  0.927778              0.0357892   
               1.00                  0.252778             0.00277778   
       0.100   0.01                  0.965278              0.0315189   
               0.10                  0.926389              0.0349989   
               1.00                  0.252778             0.00277778   
       1.000   0.01                  0.981944              0.0185821   
               0.10                  0.936111              0.0377819   
               1.00                  0.252778             0.00277778   
       10.000  0.01                    0.9875              0.0126534   
               0.10                  0.943056              0.0381629   
               1.00                  0.252778             0.00277778   
       100.000 0.01                    0.9875              0.0126534   
               0.10                  0.943056              0.0381629   
               1.00                  0.252778             0.00277778   

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                   2.09444  
       0.010   0.00                   1.18426  
       0.100   0.00                  0.418056  
       1.000   0.00                     0.275  
       10.000  0.00                     0.275  
       100.000 0.00                     0.275  
rbf    0.001   0.01                   2.00648  
               0.10                   2.02963  
               1.00                   2.64815  
       0.010   0.01                   2.17454  
               0.10                   3.88843  
               1.00                         4  
       0.100   0.01                   2.21435  
               0.10                   3.91759  
               1.00                         4  
       1.000   0.01                   1.37176  
               0.10                   3.88056  
               1.00                         4  
       10.000  0.01                   1.11389  
               0.10                   3.86898  
               1.00                         4  
       100.000 0.01                   1.11389  
               0.10                   3.86898  
               1.00                         4

*En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.